# Basic CNN implementation

## Dicoms
### Reading and pulling information from .dicom files requires nuance, and is a computationally exhaustive. Thanks to a wonderful notebook by @raddar, (http://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way), I used this formatting along with a couple other functions to properly resize and compile the image data. 

##### update 1/15:
#### functioning model, will link data preprocessing notebook once it's organized and cleaned

In [ ]:
# import necessary libraries

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import preprocessing, layers
from keras.models import Model
from tensorflow.keras.layers import *

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import os
import cv2
from PIL import Image
import pickle
from pathlib import Path

import imageio
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import os


In [ ]:
# read in data / paths

test_img = ('../input/vinbigdata-chest-xray-abnormalities-detection/test/')
train_img = ('../input/vinbigdata-chest-xray-abnormalities-detection/train/')
train_data = pd.read_csv('../input/vinbigdata-chest-xray-abnormalities-detection/train.csv')
ss = pd.read_csv('../input/vinbigdata-chest-xray-abnormalities-detection/sample_submission.csv')

In [ ]:
train_data.fillna(0, inplace=True)

In [ ]:
files = np.load('../input/unique-image-arrays/arrays.npz')

arrays = [files[key] for key in files]

In [ ]:
nps=[]

for x in arrays:
    new = np.resize(x,(256,256))
    new = np.asarray(new)
    new = keras.preprocessing.image.img_to_array(new)
    nps.append(new)

In [ ]:
class_labels = np.stack(labels)
bbox_bound = np.stack(boxes)
data = np.stack(nps)

In [ ]:
X_train, X_test, y_lab_train, y_lab_test, y_box_train, y_box_test =train_test_split(data, 
                                                                                     class_labels, 
                                                                                     bbox_bound.astype(int), 
                                                                                     test_size=0.2, 
                                                                                     shuffle=True, 
                                                                                     random_state=34)

In [ ]:
model2 = tf.keras.models.load_model('../input/cnn-model/model')

In [ ]:
model2.fit(X_train, [y_lab_train, y_box_train], epochs=5, batch_size=50, validation_split=0.2)

In [ ]:
model2.summary()

In [ ]:
cls, b = model2.predict(X_test)

In [ ]:
model2.evaluate(X_test, [y_lab_test, y_box_test])

In [ ]:
pred_labels = []

for lab in cls:
    lab = np.argmax(lab)
    pred_labels.append(lab)

In [ ]:
# for lab, box in zip(labels, b):
    

In [ ]:
tf.keras.utils.plot_model(model2)

## Next Steps...

##### for modeling notebook: 
### 1) implement KFold splitting to train & validate data 
### 2) create or manipulate sample_submission dataframe to efficiently read in output data
### 3) create IoU def to find mAP (evaluation metric)
##### current version_1.31

In [ ]:
ss.to_csv('submission.csv', index=False)